In [24]:
import numpy
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import models, transforms
from doodleLoaderSimple import DoodleDatasetSimple
from matplotlib import pyplot as plt

## Training and validation for the house image classifier

1. Cognitive computing focuses on mimicking human behaviour and reasoning to solve complex problems
2. It simulates human thought process to find solutions to complex problems 
3. They simple supplements information for human to take decisions
4. it is most likey to be used in sectors like customer service, healthcare industry etc.

1. AI augments human thinking to solve complex problems. It focuses on providing accurate results
2. AI find patterns to learn or reveal hidden information to find solutions.
3. AI is responsible for making decisions on their own minimizing the role of humans.
4. It is mostly used in finance, security, healthcare, retail, manufacturing etc.

ksi SZA 

In [ ]:
# Number to class labels mapping
class_dict = {
    0: 'stress',
    1: 'introvert',
    2: 'extrovert'
}

# Loading the data from the .csv file
# First row is a header
data = np.genfromtxt('D:\COLLEGE_STUDIES\SEM-6\ML_NLP_project\data\houseData.csv', dtype=int, delimiter=',', names=True)

print(data[:5]) 

[(0, 0) (1, 1) (2, 2) (3, 0) (4, 1)]


In [ ]:
# # Visualization: Plot the distribution of classes
# def plot_class_distribution(translation_dict):
#     """
#     Plots the distribution of class labels in the dataset.
    
#     :param translation_dict: Dictionary mapping image filenames to class labels.
#     """
#     class_counts = np.bincount(list(translation_dict.values()))  # Count occurrences of each class
#     class_labels = [class_dict[i] for i in range(len(class_counts))]  # Get class names
    
#     plt.figure(figsize=(4, 3))
#     plt.bar(class_labels, class_counts, color=['red', 'blue', 'green'])
#     plt.xlabel("Class Labels")
#     plt.ylabel("Number of Images")
#     plt.title("Class Distribution in Dataset")
#     plt.show()

# # Call the function to visualize the dataset distribution
# plot_class_distribution(translation_dict)

NameError: name 'translation_dict' is not defined

In [27]:
def count_classes(dictClass, arr):
    """
    Redundant method that counts the occurrences of each class in the dataset
    Can be used to create weights if the class distribution is unbalanced
    :param dictClass: Dictionary that maps number to class labels
    :param arr: The array that contains the data
    :return: The number of occurrences for each class in the given array
    """
    unique, count = numpy.unique(arr, return_counts=True)
    print(dict(zip(dictClass.values(), count)))
    count = 1 / count
    count = count / sum(count)
    return count

In [28]:
# Match the image IDs to the ID values in the .csv file.
translation_dict = dict( zip([f'{id}.png' for id in data['id']], data['class']))

# Prepare each image to be passed as a Tensor product to the model.
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Prepare the data by matching it to its label and transforming it to a Tensor product.
housedata = DoodleDatasetSimple(r'D:\\COLLEGE_STUDIES\\SEM-6\\ML_NLP_project\\images\\house\\', data_transforms, translation_dict)


In [ ]:
# 80% of the data for training.
train_len = int(housedata.__len__() * 0.8)
# 20% of the data for validation.
test_len = int(housedata.__len__() * 0.2 + 1)
# Split the data at a random point.
train_set, val_set = torch.utils.data.random_split(housedata, [train_len, test_len])
# Shuffle and load the labeled images in batches of 4 for training.
train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0, drop_last=True)
# Load the labeled images in batches of 4 for validation after training the model.
test_loader = DataLoader(val_set, batch_size=4, shuffle=False, num_workers=0, drop_last=True)


## Model Definition

In [30]:
class MultilabelClassifier(nn.Module):
    """
    Class that contains the layers for the model.
    Starting model ResNet-34, replace last layer with a Linear layer that outputs
    a single number, the label of the image.
    """
    def __init__(self, n_features):
        super().__init__()
        self.resnet = models.resnet34(pretrained=True)
        self.model_wo_fc = nn.Sequential(*(list(self.resnet.children())[:-1]))

        self.imageClass = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_features)
        )

    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)

        return {
            'class': self.imageClass(x)
        }
    



In [31]:
# Set the device to use as the GPU if there is compatible hardware
# Otherwise run the model on the cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultilabelClassifier(3).to(device)


c:\Users\karti\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\karti\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
def criterion(outputs, pictures):
    """
    Method used by the model as the criterion for training.
    Cross entropy loss used as the loss function
    :param outputs: Predicted labels by the model
    :param pictures: Actual labeled images from the dataset
    :return: The sum of the cross entropy loss function.
    """
    losses = 0

    for i, key in enumerate(outputs):
        loss_func = nn.CrossEntropyLoss()
        labelsTensor = pictures['class'].clone().detach()
        losses += loss_func(outputs[key], labelsTensor.long().to(device))

    return losses


In [33]:
def training(model, device, lr_rate, epochs, train_loader):
    """
    Method used by the model for training
    :param model: The model to train
    :param device: Which device to use for computation, GPU or CPU
    :param lr_rate: The learning rate used by the optimizing function
    :param epochs: How many epochs to train the model for
    :param train_loader: The loader that provides the labeled images in batches
    :return: An array containing the losses after each epoch
    """
    num_epochs = epochs
    losses = []
    checkpoint_losses = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
    n_total_steps = len(train_loader)

    for epoch in range(num_epochs):
        for i, pictures in enumerate(train_loader):
            images = pictures['image'].to(device)

            output = model(images)

            loss = criterion(output, pictures)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % (int(n_total_steps / 1)) == 0:
                checkpoint_loss = torch.tensor(losses).mean().item()
                checkpoint_losses.append(checkpoint_loss)
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {checkpoint_loss:.4f}')

    # Snippet used to save the models for inferring during runtime.
    model_save_path = r'D:\COLLEGE_STUDIES\SEM-6\ML_NLP_project\model\house\house_model_12.tar'
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': checkpoint_losses,
    }, model_save_path)



    return checkpoint_losses


In [34]:
# Call the method to train the model
checkpoint_losses = training(model, device, 0.0001, 12, train_loader)


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

In [ ]:
import torch

def validation(model, dataloader, device):
    """
    Validates the model after training.

    :param model: The trained model.
    :param dataloader: Dataloader providing labeled images in batches.
    :param device: Device (CPU or GPU) for computation.
    :return: Model accuracy in percentage.
    """
    model.eval()  # Set model to evaluation mode
    n_correct = 0
    n_samples = 0

    with torch.no_grad():
        for pictures in dataloader:
            images = pictures['image'].to(device)
            labels = pictures['class'].to(device)

            outputs = model(images)['class']  # Extract class predictions
            _, predicted = torch.max(outputs, 1)  # Get predicted class indices

            n_correct += (predicted == labels).sum().item()
            n_samples += labels.size(0)

    acc = 100.0 * n_correct / n_samples if n_samples > 0 else 0.0  # Avoid division by zero
    print(f"Validation Accuracy: {acc:.2f}%")
    
    return acc


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Ensure correct device
model.to(device)  # Move model to device

# Call the validation function
accuracy = validation(model, test_loader, device)

Validation Accuracy: 98.15%
